In [397]:
import sim 
import simConst 
import numpy as np 
import sympy as sp  

In [398]:
def connect(port): 
    sim.simxFinish(-1) #Desconecta todos los puertos. 
    clientID = sim.simxStart('127.0.0.1', port, True, True, 2000, 5)
    if clientID == 0: 
        print("Conectado a : ", port)
    else: 
        print("No se pudo conectar a: ", port)
    return clientID 
clientID = connect(1111)

Conectado a :  1111


In [399]:
ret, joint1_handle = sim.simxGetObjectHandle(clientID, "joint1", sim.simx_opmode_blocking)
ret, joint2_handle = sim.simxGetObjectHandle(clientID, "joint2", sim.simx_opmode_blocking)
ret, joint3_handle = sim.simxGetObjectHandle(clientID, "joint3", sim.simx_opmode_blocking)
ret, pos_joint0 = sim.simxGetJointPosition(clientID, joint1_handle, sim.simx_opmode_blocking)
ret, pos_joint1 = sim.simxGetJointPosition(clientID, joint2_handle, sim.simx_opmode_blocking)
ret, pos_joint3 = sim.simxGetJointPosition(clientID, joint3_handle, sim.simx_opmode_blocking)
print(f"Posicion Joint1: {pos_joint0}, Posicion Joint2: {pos_joint1}")

Posicion Joint1: 1.4056458473205566, Posicion Joint2: 0.6556954383850098


In [400]:
## Longitud de los eslabones: 
d1 = 0.1
d2 = 0.1
d3 = 0.1

q1 = np.radians(0)
q2 = np.radians(90)
q3 = np.radians(0)
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q1, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q2, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint3_handle, q3, sim.simx_opmode_blocking)
#Cinematica directa:
# Si conozco las posiciones de los angulos puedo conocer
# la pos final del robot.  
# x = d3*np.sin(q0)*np.sin(q1)
# y = -d3*np.cos(q0)*np.sin(q1)
# z = d1+d2+d3*np.cos(q1)
# print(f"Pos con cinematica en x: {x}, pos en y: {y}, pos en z: {z}") 
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_final", sim.simx_opmode_blocking)
pos_final = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
print("Posicion del extremo final: ", pos_final)

Posicion del extremo final:  (0, [0.22833967208862305, -0.002000614535063505, 0.08604872971773148])


In [401]:
#condiciones iniciales: 
p0 = {
  "x": 0, 
  "y": 0,
  "z": 0.1
}
p1 = {
  "x": 0, 
  "y": 0,
  "z": 0.2
}
p2 = {
  "x": 0, 
  "y": 0,
  "z": 0.31
}

In [402]:
#def Fabrik IK: 
#Fabrik
def fabrik(dist_vectores, segments, tgt, base):     
    base = base 
    segments = segments 
    dist_vectores = dist_vectores
    p_prima = [tgt]
    
    for i in range(len(segments) - 1): 
        eje_x = segments[i+1]["x"] - p_prima[i]["x"] 
        eje_y = segments[i+1]["y"] - p_prima[i]["y"]
        eje_z = segments[i+1]["z"] - p_prima[i]["z"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uz = eje_z/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        
        prima_x = p_prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_prima[i]["y"] + dist_vectores[i]*uy
        prima_z = p_prima[i]["z"] + dist_vectores[i]*uz
        prima = {
            "x": prima_x, 
            "y": prima_y, 
            "z": prima_z
        }
        p_prima.append(prima)

    #Backward:
    # [d1 ,d2]
    dist_vectores.reverse()
    print("p_prima:", len(p_prima))
    p_prima.reverse()
    p_2prima = [base] 
    print("Distancia de vectores: ", dist_vectores)

    for i in range(len(segments) - 1): 
        eje_x = p_prima[i+1]["x"] - p_2prima[i]["x"] 
        eje_y = p_prima[i+1]["y"] - p_2prima[i]["y"]
        eje_z = p_prima[i+1]["z"] - p_2prima[i]["z"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uz = eje_z/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        
        prima_x = p_2prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_2prima[i]["y"] + dist_vectores[i]*uy
        prima_z = p_2prima[i]["z"] + dist_vectores[i]*uz
        prima = {
            "x": prima_x, 
            "y": prima_y,
            "z": prima_z
        }
        p_2prima.append(prima)

    print("p_2prima:", p_2prima)
    
    return p_2prima   

In [403]:
#Main Fabrik
retrunCode, target_handle = sim.simxGetObjectHandle(clientID, "target", sim.simx_opmode_blocking)
returnCode, pos_target = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_blocking)
d1 = 0.1
d2 = 0.11
print("Target: ", pos_target)
tgt = {
  "x": pos_target[0], 
  "y": pos_target[1],
  "z": pos_target[2] 
}
#tgt para hacer debug: z= 0.31
# tgt = {
#   "x": 0, 
#   "y": 0,
#   "z": 0.31 
# }

dist_vectores = [d2, d1]
segments = [p2, p1, p0] 
base = segments[len(segments) - 1]

for i in range(2): 
  p_2prima = fabrik(dist_vectores, segments, tgt, base)
  segments = p_2prima 
  dist_vectores = [d2, d1]
  print("Segments news: ", segments)

Target:  [0.024999983608722687, 0.1499999761581421, 0.12499997764825821]
p_prima: 3
Distancia de vectores:  [0.1, 0.11]
p_2prima: [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0.009651409026581827, 'y': 0.05790848292295012, 'z': 0.1809534304978116}, {'x': 0.02516286901301063, 'y': 0.1509772890693064, 'z': 0.1244061765275137}]
Segments news:  [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0.009651409026581827, 'y': 0.05790848292295012, 'z': 0.1809534304978116}, {'x': 0.02516286901301063, 'y': 0.1509772890693064, 'z': 0.1244061765275137}]
p_prima: 3
Distancia de vectores:  [0.1, 0.11]
p_2prima: [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0.009497483413127346, 'y': 0.056984928783488854, 'z': 0.18162423476124034}, {'x': 0.025000987496944863, 'y': 0.15000599949046697, 'z': 0.12499631085747687}]
Segments news:  [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0.009497483413127346, 'y': 0.056984928783488854, 'z': 0.18162423476124034}, {'x': 0.025000987496944863, 'y': 0.15000599949046697, 'z': 0.12499631085747687}]


In [404]:
#angulos FABRIK 
def angulos_fabrik(p_2prima): 
    print(f"target: {tgt} \n")
    print("p_2prima", p_2prima)
    x = p_2prima[1]["x"]
    y = p_2prima[1]["y"]
    z = p_2prima[1]["z"]
    print(f"p_prima 2do punto: x:{x}, y:{y}, z: {z}")
    
    #Primer eslabon: 
    q1 = np.arctan2(y,x)
    eje_x1 = 0
    eje_y1 = 0
    eje_z1 = d1 

    #Cinematica directa segundo eslabon:
    success = 0 
    for q2 in np.arange(0, np.pi*2, 0.0001): 
        if (abs(d1 + d2*np.cos(q2) - z) < 0.001): 
            print("Se encontro un angulo ")
            success = 1
            break
    if (success == 1): 
        print(f"Se encontro el angulo: {q2}")
    else: 
        print("No se encontro un angulo")    
    eje_x2 = -d2*np.cos(q1)*np.sin(q2) 
    eje_y2 = -d2*np.sin(q1)*np.sin(q2)
    eje_z2 = d1 + d2*np.cos(q2)
    print(f"eje x: {eje_x2}, eje y: {eje_y2}, eje z: {eje_z2} \n")

    #Cinematica directa tercer eslabon:
    x = p_2prima[2]["x"]
    y = p_2prima[2]["y"]
    z = p_2prima[2]["z"]
    print(f"p_prima 3er punto: x:{x}, y:{y}, z: {z}")
    success = 0  
    for q3 in np.arange(0, np.pi*2, 0.0001): 
        if (abs(d1 + d3*np.cos(q2 + q3) + d2*np.cos(q2) - z) < 0.001): 
            print("Se encontro un angulo ")
            success = 1
            break
    eje_y3 = np.sin(q1)*(d3*np.sin(q2 + q3) + d2*np.sin(q2))
    eje_z3 = d1 + d3*np.cos(q2 + q3) + d2*np.cos(q2)
    print(f"eje y eslabon3: {eje_y3}, eje z 3er eslabon: {eje_z3}")
    if (success == 1): 
        print(f"Se encontro el angulo: {q2}")
    else: 
        print("No se encontro un angulo")
    print(f"q1: {q1}, q2: {q2}, q3: {q2}")
    return [q1, q2, q3]

In [405]:
#Calculo de los angulos entre los eslabones: 
q = angulos_fabrik(p_2prima)

#Movimiento de los motores:  
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q[0], sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q[1], sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint3_handle, q[2], sim.simx_opmode_blocking)

target: {'x': 0.024999983608722687, 'y': 0.1499999761581421, 'z': 0.12499997764825821} 

p_2prima [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0.009497483413127346, 'y': 0.056984928783488854, 'z': 0.18162423476124034}, {'x': 0.025000987496944863, 'y': 0.15000599949046697, 'z': 0.12499631085747687}]
p_prima 2do punto: x:0.009497483413127346, y:0.056984928783488854, z: 0.18162423476124034
Se encontro un angulo 
Se encontro el angulo: 0.7211000000000001
eje x: -0.011939181095847616, eje y: -0.07163512215664139, eje z: 0.18261879464361375 

p_prima 3er punto: x:0.025000987496944863, y:0.15000599949046697, z: 0.12499631085747687
eje y eslabon3: 0.1367516401701432, eje z 3er eslabon: 0.25773242157691784
No se encontro un angulo
q1: 1.4056477299271009, q2: 0.7211000000000001, q3: 0.7211000000000001
